### Probabilistic Language Modeling

> Language models offer a way to assign a probability to a sentence or other sequence of words, and to predict a word from preceding words.

**The chain rule of probability**

$P(w_1,w_2,\ldots,w_n) = \prod P(w_i \mid w_{1}, w_{2}, \ldots, w_{i-1})$

**Markov assumption**

$P(w_{i} \mid w_1, w_2, \ldots, w_{i-1}) \approx P(w_{i} \mid w_{i-k}, \ldots,w_{i-1})$

#### N-Grams

> n-grams are Markov models that estimate words from a fixed window of previous words. n-gram probabilities can be estimated by counting in a corpus and normalizing (the **maximum likelihood estimate**).

In [1]:
import pandas as pd
import numpy as np

import string
import re

In [203]:
df = pd.read_csv('/content/berkeley_restaurant.csv', index_col=0)

In [204]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [205]:
from nltk.tokenize import word_tokenize

In [239]:
def padded_tokens(text):
  prefix, suffix = '<s>', '</s>'
  return [prefix] + word_tokenize(text) + [suffix]

In [240]:
df['tokens'] = df['text'].apply(lambda text: padded_tokens(text))

In [241]:
from nltk.util import ngrams

In [242]:
df['unigram'] = df['tokens'].apply(lambda tokens: list((ngrams(tokens, 1))))

In [243]:
df['bigram'] = df['tokens'].apply(lambda tokens: list((ngrams(tokens, 2))))

In [244]:
df.head()

,text,tokens,unigram,bigram
0,okay lets see i want to go to a thai restauran...,"[<s>, okay, lets, see, i, want, to, go, to, a,...","[(<s>,), (okay,), (lets,), (see,), (i,), (want...","[(<s>, okay), (okay, lets), (lets, see), (see,..."
1,i like to eat uh i like to eat at lunch time s...,"[<s>, i, like, to, eat, uh, i, like, to, eat, ...","[(<s>,), (i,), (like,), (to,), (eat,), (uh,), ...","[(<s>, i), (i, like), (like, to), (to, eat), (..."
2,i dont want to walk for more than five minutes,"[<s>, i, dont, want, to, walk, for, more, than...","[(<s>,), (i,), (dont,), (want,), (to,), (walk,...","[(<s>, i), (i, dont), (dont, want), (want, to)..."
3,tell me more about the uh na nakapan uh restau...,"[<s>, tell, me, more, about, the, uh, na, naka...","[(<s>,), (tell,), (me,), (more,), (about,), (t...","[(<s>, tell), (tell, me), (me, more), (more, a..."
4,i like to go to a hamburger restaurant,"[<s>, i, like, to, go, to, a, hamburger, resta...","[(<s>,), (i,), (like,), (to,), (go,), (to,), (...","[(<s>, i), (i, like), (like, to), (to, go), (g..."


**Maximum Likelihood Estimate**

$P(W_i \mid W_{i-1}) = \frac{count(W_{i-1}, W_i)}{count(W_{i-1})}$

In [245]:
from collections import Counter

In [246]:
unigram_counts = Counter(df['unigram'].explode())

In [247]:
bigram_counts = Counter(df['bigram'].explode())

In [248]:
def bigram_proba(bigram_counts, unigram_counts):
  proba = {}
  for bigram in bigram_counts.keys():
    try:
      prev_word, word = bigram
      proba[bigram] = bigram_counts.get(bigram) / unigram_counts.get((prev_word,))
    except:
      continue

  return proba

In [249]:
proba = bigram_proba(bigram_counts, unigram_counts)

In [250]:
sample_txt = 'i want english food'

**Raw Probabilities**

$P(W_1, W_2, \ldots, W_n) = P(W_2 \mid W_1) \times \ldots \times P(W_n \mid W_{n-1})  $

In [251]:
def predict_proba(text):
  bigrams = list(ngrams(text.split(), 2))
  preds = 1
  for bigram in bigrams:
    preds *= proba.get(bigram, 0)
    
  return preds

**Log Probabilities**

$P(W_1, W_2, \ldots, W_n) = exp(log(P(W_2 \mid W_1)) + \ldots + log(P(W_n \mid W_{n-1})))  $

In [252]:
import math

In [254]:
def predict_log_proba(text):
  bigrams = list(ngrams(text.split(), 2))
  preds = 0
  for bigram in bigrams:
    preds += math.log(proba.get(bigram, 0))
  
  return math.exp(preds)

In [256]:
predict_log_proba(sample_txt)

0.000154072590536549

**Perplexity**

The inverse probability of the test set, normalized
by the number of words

$PP(W) = P(W_1, W_2, \ldots, W_n)^{-\frac{1}{N}}$

$PP(W) = \sqrt[N]{\prod\frac{1}{P(W_i \mid W_1 \ldots W_{i-1})}}$

In [257]:
def perplexity(text):
  n = len(list(ngrams(text.split(), 2)))
  p = predict_proba(text)

  return math.pow(1/p, 1/n)

In [258]:
perplexity(sample_txt)

18.653408668710675

**Generalization and Zeros**
- Out of Vocabulary (OOV)

**Smoothing**

> Smoothing algorithms provide a more sophisticated way to estimate the probability of n-grams. Commonly used smoothing algorithms for n-grams rely on lower-order n-gram counts through backoff or interpolation

In [259]:
def bigram_smooth_proba(bigram_counts, unigram_counts, smoothing='laplace'):
  proba = {}
  for bigram in bigram_counts.keys():
    try:
      V = len(unigram_counts.keys())
      prev_word, word = bigram
      if smoothing == 'laplace':
        proba[bigram] = (bigram_counts.get(bigram) + 1) / (unigram_counts.get((prev_word,)) + V)
      elif smoothing == 'add_k':
        k = 0.1
        proba[bigram] = (bigram_counts.get(bigram) + k) / (unigram_counts.get((prev_word,)) + (k * V))
    except:
      continue

  return proba

**Laplace Smoothing**

$P_{Laplace}(W_i \mid W_{i-1}) = \frac{count(W_{i-1}, W_i) \: + \: 1}{count(W_{i-1}) \: + \: V}$

In [260]:
laplace_proba = bigram_smooth_proba(bigram_counts, unigram_counts, smoothing='laplace')

**Add-k Smoothing**

$P_{Add-k}(W_i \mid W_{i-1}) = \frac{count(W_{i-1}, W_i) \: + \: k}{count(W_{i-1}) \: + \: kV}$

In [261]:
add_k_proba = bigram_smooth_proba(bigram_counts, unigram_counts, smoothing='add_k')

In [262]:
def predict_smooth_proba(text, smoothing='laplace'):
  bigrams = list(ngrams(text.split(), 2))
  preds = 1
  for bigram in bigrams:
    if smoothing == 'laplace':
      preds *= laplace_proba.get(bigram, 0)
    elif smoothing == 'add_k':
      preds *= add_k_proba.get(bigram, 0)

  return preds

In [263]:
predict_smooth_proba(sample_txt, smoothing='laplace')

1.941694493179284e-07

In [264]:
predict_smooth_proba(sample_txt, smoothing='add_k')

1.8859943386849113e-06

**Other Smoothing**
- Stupid Backoff (For very large N-grams like web)
- Kneser-Ney smoothing

**Perplexity’s Relation to Entropy**

**Language Modeling Toolkits**
- SRILM
- KenLM